In [2]:
import cv2
import json
import numpy as np

In [ ]:
class ImageTransformer(object):

    def __init__(self, image):
        self.image = image

        self.height = self.image.shape[0]
        self.width = self.image.shape[1]
        self.num_channels = self.image.shape[2]

    """ Wrapper of Rotating a Image """

    def rotate_along_axis(self, theta=0, phi=0, gamma=0, dx=0, dy=0, dz=0):
        # Get ideal focal length on z axis
        # NOTE: Change this section to other axis if needed
        d = np.sqrt(self.height ** 2 + self.width ** 2)
        self.focal = d / (2 * np.sin(gamma) if np.sin(gamma) != 0 else 1)
        dz = self.focal

        # Get projection matrix
        mat = self.get_M(theta, phi, gamma, dx, dy, dz)

        return cv2.warpPerspective(self.image.copy(), mat, (self.width, self.height))

    """ Get Perspective Projection Matrix """

    def get_M(self, theta, phi, gamma, dx, dy, dz):
        w = self.width
        h = self.height
        f = self.focal

        # Projection 2D -> 3D matrix
        A1 = np.array([[1, 0, -w / 2],
                       [0, 1, -h / 2],
                       [0, 0, 1],
                       [0, 0, 1]])

        # Rotation matrices around the X, Y, and Z axis
        RX = np.array([[1, 0, 0, 0],
                       [0, np.cos(theta), -np.sin(theta), 0],
                       [0, np.sin(theta), np.cos(theta), 0],
                       [0, 0, 0, 1]])

        RY = np.array([[np.cos(phi), 0, -np.sin(phi), 0],
                       [0, 1, 0, 0],
                       [np.sin(phi), 0, np.cos(phi), 0],
                       [0, 0, 0, 1]])

        RZ = np.array([[np.cos(gamma), -np.sin(gamma), 0, 0],
                       [np.sin(gamma), np.cos(gamma), 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])

        # Composed rotation matrix with (RX, RY, RZ)
        R = np.dot(np.dot(RX, RY), RZ)

        # Translation matrix
        T = np.array([[1, 0, 0, dx],
                      [0, 1, 0, dy],
                      [0, 0, 1, dz],
                      [0, 0, 0, 1]])

        # Projection 3D -> 2D matrix
        A2 = np.array([[f, 0, w / 2, 0],
                       [0, f, h / 2, 0],
                       [0, 0, 1, 0]])

        # Final transformation matrix
        return np.dot(A2, np.dot(T, np.dot(R, A1)))

    
def sn(s):
    return s['secs'] + s['nsecs'] / 1000000000

def explore(dat, stats):
    initial = cv2.imread('data/' + dat['image_initial']['filename'])
    final = cv2.imread('data/' + dat['image_final']['filename'])   
    
    # Find optimal phi based on timestamps and imu readings

    phi_average = stats['delta_t'] * stats['avg_imu_z']
    
def stats(dat):
    initial_t = sn(dat['image_initial']['stamp'])
    final_t = sn(dat['image_final']['stamp'])
        
    delta_t = final_t - initial_t
    
    imu_stamps = np.zeros((len(dat['last_imu']),))
    imu_av = np.zeros((len(dat['last_imu']),))
    
    for idx, imu in enumerate(dat['last_imu']):
        
        imu_av[idx] = imu['angular_velocity']['z']
        imu_stamps[idx] = sn(imu['stamp'])
                                                 
    
    avg_imu_t = np.average(imu_stamps)
    max_imu_t = np.amax(imu_stamps)
    min_imu_t = np.amin(imu_stamps)
    mid_imu_t = np.median(imu_stamps)
    
    avg_imu_z = np.average(imu_av)
    max_imu_z = np.amax(imu_av)
    min_imu_z = np.amin(imu_av)
    mid_imu_z = np.median(imu_av)    
    
    return {'initial_t': initial_t,
           'final_t': final_t,
           'delta_t': delta_t,
           'avg_imu_t': avg_imu_t,
           'max_imu_t': max_imu_t,
           'min_imu_t': min_imu_t,
           'med_imu_t': med_imu_t,
           'avg_imu_z': avg_imu_z,
           'max_imu_z': max_imu_z,
           'min_imu_z': min_imu_z,
           'mid_imu_z': mid_imu_z}
        
    

In [ ]:
files = open('data/list.txt', 'r').read().split("\n")

for file in files:
    dat = json.loads(open('data/' + file, 'r').read())
    s = process(dat)
    explore(s, dat)